In [81]:
import requests
import pandas as pd
import re


In [82]:
# Fetch data from API - get ALL records
API_URL = "https://data.iledefrance.fr/api/records/1.0/search/"

# First, get the total number of records
params_count = {
    "dataset": "aides-appels-a-projets",
    "rows": 1  # Just get count
}

r_count = requests.get(API_URL, params=params_count, timeout=30)
r_count.raise_for_status()
data_count = r_count.json()
total_records = data_count.get('nhits', 0)
print(f"📊 Total records available: {total_records}")

# Now fetch all records with pagination
all_records = []
rows_per_page = 100  # Max rows per request
num_pages = (total_records // rows_per_page) + (1 if total_records % rows_per_page else 0)

print(f"📥 Fetching {num_pages} pages of {rows_per_page} records each...")

for page in range(num_pages):
    start = page * rows_per_page
    params = {
        "dataset": "aides-appels-a-projets",
        "rows": rows_per_page,
        "start": start
    }
    
    try:
        r = requests.get(API_URL, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        all_records.extend(data.get('records', []))
        print(f"  ✅ Page {page + 1}/{num_pages}: {len(data.get('records', []))} records")
    except Exception as e:
        print(f"  ❌ Error fetching page {page + 1}: {str(e)}")
        continue

print(f"\n✅ Successfully fetched {len(all_records)} records total")

# Store in data variable for backward compatibility
data = {"records": all_records}


📊 Total records available: 343
📥 Fetching 4 pages of 100 records each...
  ✅ Page 1/4: 100 records
  ✅ Page 2/4: 100 records
  ✅ Page 3/4: 100 records
  ✅ Page 4/4: 43 records

✅ Successfully fetched 343 records total


In [83]:
# Create DataFrame from API data
rows = []
for rec in data["records"]:
    row = rec["fields"].copy()
    row["_recordid"] = rec["recordid"]
    row["_record_timestamp"] = rec["record_timestamp"]
    row["_record_url"] = rec.get("record_url")
    rows.append(row)

df = pd.DataFrame(rows)
print(f"DataFrame shape: {df.shape}")


DataFrame shape: (343, 39)


In [84]:
# Clean HTML entities and tags
def clean_text(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r"<[^>]+>", "", text)
    html_entities = {
        "&eacute;": "é", "&icirc;": "î", "&agrave;": "à",
        "&nbsp;": " ", "&quot;": "\"", "&amp;": "&",
        "&rsquo;": "'", "&ldquo;": "\u201c", "&rdquo;": "\u201d"
    }
    for entity, char in html_entities.items():
        text = text.replace(entity, char)
    return text

df['entete'] = df['entete'].apply(clean_text)


In [85]:
# Extract maximum monetary amount from 'modalite_txt'
def extract_max_amount(text):
    if not isinstance(text, str):
        return None
    
    text = text.replace("\xa0", " ")  # Non-breaking space
    
    # Look for amounts like: 35.000€, 35,000€, 35 000€, 35€
    amounts = re.findall(r"\b(\d+[\s.,]*\d*)\s*(?:€|euros?|EUROS?)", text)
    
    if not amounts:
        return None
    
    cleaned_amounts = []
    for amt in amounts:
        # Remove spaces
        amt = amt.replace(" ", "")
        
        # Check if it contains a comma (European decimal separator)
        if "," in amt:
            # "1.234,56" → "1234.56" (European format)
            amt = amt.replace(".", "").replace(",", ".")
        else:
            # "1.234" or "1234" → "1234"
            # If it has a dot and 3+ digits after, it's a thousands separator
            parts = amt.split(".")
            if len(parts) > 1 and len(parts[-1]) == 3 and parts[-1].isdigit():
                # "1.234" format - dot is thousands separator
                amt = amt.replace(".", "")
            # else: "1234.56" format - dot is decimal separator, keep it
        
        try:
            cleaned_amounts.append(float(amt))
        except ValueError:
            continue
    
    return max(cleaned_amounts) if cleaned_amounts else None

df['montant_max'] = df['modalite_txt'].apply(extract_max_amount)


In [86]:
# Extract emails and phone numbers from contact field
def extract_contact_info(text):
    if not isinstance(text, str):
        return ""
    emails = re.findall(r"[\w.-]+@[\w.-]+", text)
    phones = re.findall(r"\+?\d[\d\s.-]{7,}\d", text)
    return ", ".join(emails + phones)

df['contact'] = df['contact'].apply(extract_contact_info)


In [87]:
# Simple listing of columns
print("Colonnes de mapped_df:")
print(list(df.columns))
#df.head(5)

Colonnes de mapped_df:
['qui_peut_en_beneficier', 'chapo_txt', 'id_theme', 'porteur_aide', 'id_aide', 'id_publics', 'presence_ref', 'qui', 'entete', 'modalite_txt', 'competences', 'demarches', 'demarches_txt', 'date_cloture', 'mots_cles', 'nom_de_l_aide_de_la_demarche', 'recup_id_thm', 'porteur_siren', 'objectif', 'url_descriptif', 'publicsbeneficiaireprecision', 'theme', 'demarche_en_ligne', 'objectif_txt', 'modalite', 'reference_administrative', 'date_ouverture', 'date', '_recordid', '_record_timestamp', '_record_url', 'notes_txt', 'deliberation_cadre', 'engagements_du_beneficiaire', 'notes', 'contact', 'titre_alternatif_de_l_aide', 'libelle', 'datedebutfuturcampagne', 'montant_max']


In [88]:
# Map columns to final schema
# Format: 'new_column_in_mapped_df': 'source_column_in_df'

# First, check what columns are in df
print("📋 Colonnes disponibles dans df:")
print(list(df.columns))
print("\n")

# Create mapped DataFrame with proper mapping
mapped_df = pd.DataFrame()

# Map columns from df to mapped_df (mapped_df_name: df_name)
if '_recordid' in df.columns:
    mapped_df['id_record'] = df['_recordid']
if 'chapo_txt' in df.columns:
    mapped_df['titre'] = df['chapo_txt']
if 'porteur_aide' in df.columns:
    mapped_df['organisme'] = df['porteur_aide']
if 'date_ouverture' in df.columns:
    mapped_df['date_publication'] = df['date_ouverture']
if 'date_cloture' in df.columns:
    mapped_df['date_limite'] = df['date_cloture']
if 'theme' in df.columns:
    mapped_df['categories'] = df['theme']
if 'qui' in df.columns:
    mapped_df['public_cible'] = df['qui']
if 'montant_max' in df.columns:
    mapped_df['montant_max'] = df['montant_max']
if 'rate_percent' in df.columns:
    mapped_df['taux_financement'] = df['rate_percent']
if 'url_descriptif' in df.columns:
    mapped_df['url_source'] = df['url_descriptif']
if 'contact' in df.columns:
    mapped_df['contact'] = df['contact']
if 'modalite_txt' in df.columns:
    mapped_df['modalite'] = df['modalite_txt']
if 'demarches_txt' in df.columns:
    mapped_df['demarches'] = df['demarches_txt']
if 'mots_cles' in df.columns:
    mapped_df['mots_cles'] = df['mots_cles']
if 'nom_de_l_aide_de_la_demarche' in df.columns:
    mapped_df['nom_de_l_aide_de_la_demarche'] = df['nom_de_l_aide_de_la_demarche']
if 'objectif_txt' in df.columns:
    mapped_df['objectif'] = df['objectif_txt']
if 'notes_txt' in df.columns:
    mapped_df['note'] = df['notes_txt']

# Add hardcoded values
mapped_df['tags'] = None
mapped_df['perimetre_geo'] = "IDF"

print(f"✅ Mapped DataFrame created with {len(mapped_df)} rows and {len(mapped_df.columns)} columns")
print(f"📊 Colonnes dans mapped_df: {list(mapped_df.columns)}")


📋 Colonnes disponibles dans df:
['qui_peut_en_beneficier', 'chapo_txt', 'id_theme', 'porteur_aide', 'id_aide', 'id_publics', 'presence_ref', 'qui', 'entete', 'modalite_txt', 'competences', 'demarches', 'demarches_txt', 'date_cloture', 'mots_cles', 'nom_de_l_aide_de_la_demarche', 'recup_id_thm', 'porteur_siren', 'objectif', 'url_descriptif', 'publicsbeneficiaireprecision', 'theme', 'demarche_en_ligne', 'objectif_txt', 'modalite', 'reference_administrative', 'date_ouverture', 'date', '_recordid', '_record_timestamp', '_record_url', 'notes_txt', 'deliberation_cadre', 'engagements_du_beneficiaire', 'notes', 'contact', 'titre_alternatif_de_l_aide', 'libelle', 'datedebutfuturcampagne', 'montant_max']


✅ Mapped DataFrame created with 343 rows and 18 columns
📊 Colonnes dans mapped_df: ['id_record', 'titre', 'organisme', 'date_publication', 'date_limite', 'categories', 'public_cible', 'montant_max', 'url_source', 'contact', 'modalite', 'demarches', 'mots_cles', 'nom_de_l_aide_de_la_demarche', 

In [89]:
# Convert fields with ||| separator to lists for Airtable Multiple Select

def convert_pipe_to_list(value):
    """Convert string with ||| separator to list"""
    if pd.isna(value) or value == '' or value is None:
        return None  # Return None instead of empty list
    if isinstance(value, str):
        # Split by ||| and remove empty strings
        items = [item.strip() for item in value.split('|||') if item.strip()]
        return items if items else None  # Return None if empty
    return None

# Convert public_cible to list (Multiple Select in Airtable)
if 'public_cible' in mapped_df.columns:
    mapped_df['public_cible'] = mapped_df['public_cible'].apply(convert_pipe_to_list)
    print(f"✅ Converted 'public_cible' to list format (Multiple Select)")
    non_null = mapped_df['public_cible'].dropna()
    print(f"   Exemple: {non_null.iloc[0] if len(non_null) > 0 else 'N/A'}")

# Convert categories to list (Multiple Select in Airtable)
if 'categories' in mapped_df.columns:
    has_separator = mapped_df['categories'].astype(str).str.contains('\|\|\|').any()
    if has_separator:
        mapped_df['categories'] = mapped_df['categories'].apply(convert_pipe_to_list)
        print(f"✅ Converted 'categories' to list format (Multiple Select)")
        non_null = mapped_df['categories'].dropna()
        print(f"   Exemple: {non_null.iloc[0] if len(non_null) > 0 else 'N/A'}")
    else:
        print(f"⚠️ 'categories' does not contain ||| separator, keeping as text")

# Convert mots_cles to list (Multiple Select in Airtable)
if 'mots_cles' in mapped_df.columns:
    has_separator = mapped_df['mots_cles'].astype(str).str.contains('\|\|\|').any()
    if has_separator:
        mapped_df['mots_cles'] = mapped_df['mots_cles'].apply(convert_pipe_to_list)
        print(f"✅ Converted 'mots_cles' to list format (Multiple Select)")
        non_null = mapped_df['mots_cles'].dropna()
        print(f"   Exemple: {non_null.iloc[0] if len(non_null) > 0 else 'N/A'}")
    else:
        print(f"⚠️ 'mots_cles' does not contain ||| separator, keeping as text")

print(f"\n✅ Multiple Select fields prepared for Airtable")
print(f"   - public_cible: Multiple Select (list)")
print(f"   - categories: Multiple Select (list) if contains |||")
print(f"   - mots_cles: Multiple Select (list) if contains |||")
print(f"   (Empty values converted to None for compatibility)")

✅ Converted 'public_cible' to list format (Multiple Select)
   Exemple: ["Établissement d'enseignement supérieur"]
✅ Converted 'categories' to list format (Multiple Select)
   Exemple: ['Éducation & recherche : Recherche']
✅ Converted 'mots_cles' to list format (Multiple Select)
   Exemple: ['Attractivité']

✅ Multiple Select fields prepared for Airtable
   - public_cible: Multiple Select (list)
   - categories: Multiple Select (list) if contains |||
   - mots_cles: Multiple Select (list) if contains |||
   (Empty values converted to None for compatibility)


<>:22: SyntaxWarning: invalid escape sequence '\|'
<>:33: SyntaxWarning: invalid escape sequence '\|'
<>:22: SyntaxWarning: invalid escape sequence '\|'
<>:33: SyntaxWarning: invalid escape sequence '\|'
C:\Users\WALID\AppData\Local\Temp\ipykernel_28640\335471511.py:22: SyntaxWarning: invalid escape sequence '\|'
  has_separator = mapped_df['categories'].astype(str).str.contains('\|\|\|').any()
C:\Users\WALID\AppData\Local\Temp\ipykernel_28640\335471511.py:33: SyntaxWarning: invalid escape sequence '\|'
  has_separator = mapped_df['mots_cles'].astype(str).str.contains('\|\|\|').any()


In [90]:
# Simple listing of columns
print("Colonnes de mapped_df:")
print(list(mapped_df.columns))

Colonnes de mapped_df:
['id_record', 'titre', 'organisme', 'date_publication', 'date_limite', 'categories', 'public_cible', 'montant_max', 'url_source', 'contact', 'modalite', 'demarches', 'mots_cles', 'nom_de_l_aide_de_la_demarche', 'objectif', 'note', 'tags', 'perimetre_geo']


In [91]:
# Display with HTML rendering
#from IPython.display import HTML
#HTML(mapped_df.to_html(escape=False))
mapped_df.head(5)

,id_record,titre,organisme,date_publication,date_limite,categories,public_cible,montant_max,url_source,contact,modalite,demarches,mots_cles,nom_de_l_aide_de_la_demarche,objectif,note,tags,perimetre_geo
0,db22f057f603f4142f40ea44dd5d2a9f1a117ba7,"La Région entend, via ces distinctions, permet...",Région Île-de-France,2023-04-11T22:00:00+00:00,2023-09-04T22:00:00+00:00,[Éducation & recherche : Recherche],[Établissement d'enseignement supérieur],350000.0,https://www.iledefrance.fr/aides-et-appels-a-p...,,Les projets présentés par un établissement ou ...,Les projets sont à déposer sur la plateforme d...,[Attractivité],Trophées franciliens de l'innovation numérique...,Les projets qui s'inscrivent dans les stratégi...,NaN,None,IDF
1,4a4528ff2716861a594f3cff2baba631a4193772,Dans le cadre de son programme Région solidair...,Région Île-de-France,NaN,NaN,[Santé & Social : Action sociale],"[Association - Fondation, Association - ONG, A...",300000.0,https://www.iledefrance.fr/aides-et-appels-a-p...,"solidarites@iledefrance.fr, solidarites@iledef...",Pour les services et accueils de jour des femm...,Transmettez votre dossier via la plateforme de...,[Région solidaire],"Soutien à l'hébergement, aux services et accue...",Sont concernés les établissements d’hébergemen...,Pour toute subvention votée par le Conseil rég...,None,IDF
2,f744974f52c3b4ca051fa22225e4f35d22495e37,La Région Île-de-France promeut l’égal accès a...,Région Île-de-France,NaN,NaN,[Santé & Social : Santé],[Collectivité ou institution - Communes de 10 ...,300000.0,https://www.iledefrance.fr/aides-et-appels-a-p...,"magdouda.bendjebla@iledefrance.fr, sante@ilede...",Aide en investissement pour les travaux de cré...,La transmission du dossier se fera via la plat...,[Région solidaire],Aide à l'investissement en faveur de l'accès à...,"Travaux de création, extension et restructurat...",Les projets doivent démarrer à partir du vote ...,None,IDF
3,2aa9899392fdcaba4a5d6fc9757f4293b4304fde,"Avec ce dispositif, la Région, en partenariat ...",Région Île-de-France,NaN,NaN,[Pas dans la liste proposée],[Entreprises],650000.0,https://www.iledefrance.fr/aides-et-appels-a-p...,,"Prêt sans garantie ni caution personnelle, d’u...",Pour déposer une demande de prêt il convient d...,[Attractivité],Contrat de développement transmission,Le contrat de développement transmission vise ...,(1) Acquéreur et cible doivent être une PME :...,None,IDF
4,8009bd388ed82bf8f508f8d48915f2d8173e07de,Le parcours a pour objectifs principaux de dév...,Région Île-de-France,2023-08-31T22:00:00+00:00,2024-06-29T22:00:00+00:00,[Éducation & recherche : Lycée],"[Particulier - Lycéen, Établissement d'enseign...",NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,"direction@lesideateurs-france.org, direction@l...",Action gratuite pour les établissements franci...,"Pour bénéficier de ce programme, s’inscrire di...",None,LES IDEATEURS : Programme « Parcours1-CREA : L...,"L'expérience entrepreneuriale, soit la créatio...",NaN,None,IDF


## Upload to Airtable

⚠️ Assurez-vous d'avoir configuré le fichier `.env` avec vos credentials Airtable avant d'exécuter cette cellule.

In [92]:
# Upload to Airtable
import sys
sys.path.append('..')

from connectors.airtable_connector import AirtableConnector

# Initialize Airtable connector
airtable = AirtableConnector()

# Optional: Uncomment to clear existing data before upload
airtable.clear_table()

# Upload DataFrame
uploaded = airtable.upload_dataframe(mapped_df)
print(f"\n🎉 Successfully uploaded {uploaded} records to Airtable!")

🗑️ Deleted batch 1/35
🗑️ Deleted batch 2/35
🗑️ Deleted batch 3/35
🗑️ Deleted batch 4/35
🗑️ Deleted batch 5/35
🗑️ Deleted batch 6/35
🗑️ Deleted batch 7/35
🗑️ Deleted batch 8/35
🗑️ Deleted batch 9/35
🗑️ Deleted batch 10/35
🗑️ Deleted batch 11/35
🗑️ Deleted batch 12/35
🗑️ Deleted batch 13/35
🗑️ Deleted batch 14/35
🗑️ Deleted batch 15/35
🗑️ Deleted batch 16/35
🗑️ Deleted batch 17/35
🗑️ Deleted batch 18/35
🗑️ Deleted batch 19/35
🗑️ Deleted batch 20/35
🗑️ Deleted batch 21/35
🗑️ Deleted batch 22/35
🗑️ Deleted batch 23/35
🗑️ Deleted batch 24/35
🗑️ Deleted batch 25/35
🗑️ Deleted batch 26/35
🗑️ Deleted batch 27/35
🗑️ Deleted batch 28/35
🗑️ Deleted batch 29/35
🗑️ Deleted batch 30/35
🗑️ Deleted batch 31/35
🗑️ Deleted batch 32/35
🗑️ Deleted batch 33/35
🗑️ Deleted batch 34/35
🗑️ Deleted batch 35/35
✅ Cleared 343 records from table
✅ Converted date_publication to ISO date format
✅ Converted date_limite to ISO date format
✅ Converted montant_max to numeric format
⚠️ Could not verify fields, uploading 

In [93]:
# Force reload environment variables
import os
from dotenv import load_dotenv

# Clear existing environment variables
for key in ['AIRTABLE_TOKEN', 'AIRTABLE_BASE_ID', 'AIRTABLE_TABLE_NAME']:
    os.environ.pop(key, None)

# Reload .env file
load_dotenv(override=True)

# Verify new values are loaded
print("✅ Variables d'environnement rechargées:")
print(f"AIRTABLE_BASE_ID: {os.getenv('AIRTABLE_BASE_ID')}")
print(f"AIRTABLE_TABLE_NAME: {os.getenv('AIRTABLE_TABLE_NAME')}")
print(f"AIRTABLE_API_KEY: {'***' + os.getenv('AIRTABLE_TOKEN', '')[-4:] if os.getenv('AIRTABLE_TOKEN') else 'NON DÉFINIE'}")

✅ Variables d'environnement rechargées:
AIRTABLE_BASE_ID: appRgVRJoIZUC19C4
AIRTABLE_TABLE_NAME: tblBMgHqNQ9iivWUh
AIRTABLE_API_KEY: ***7a92
